In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras import layers

In [8]:
(train_ds, test_ds), ds_info = tfds.load('cars196', split = ["train", "test"], as_supervised= True, with_info = True)

In [11]:
height, width = 150, 150
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, (height, width)), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, (height, width)), y))

In [12]:
batch_size = 32
def augment_func(image,label):
    image = tf.image.resize_with_crop_or_pad(image,height+6,width+6)
    #image = tf.clip_by_value(image,0,255) #make sure you have no color value higher than 225 or lower than 0.
    image = tf.image.random_crop(image,size=[height,width,3])
    image = tf.image.random_flip_left_right(image) #different aspect of vehicles
    image = tf.image.random_hue(image,0.2) #random color, change a red cat into a blue car
    image = tf.image.random_contrast(image,0.5,2)# random contrast
    image = tf.image.random_saturation(image,0,2)# random sturations
    return image, label


train_ds = train_ds.cache().map(augment_func).shuffle(100).batch(batch_size).prefetch(buffer_size=10) # cache makes the images ready before running
test_ds = test_ds.cache().map(augment_func).batch(batch_size).prefetch(buffer_size=10)

In [13]:
base_model = tf.keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(height, width, 3),
    include_top=False, # Do not include the final ImageNet classifier layer at the top.
)  

base_model.trainable = False # We want to update all the model weights, so set this to true.

# Create new model on surrounding our pretrained base model.
inputs = tf.keras.Input(shape=(height, width, 3))

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(inputs)
norm_layer.set_weights([mean, var,1])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x) # this is a neural network operation to help adapt the features learned by the pretrained model to our specific task.
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
num_outputs = ds_info.features['label'].num_classes # This is the number of output variables we want, 196 in this case.
outputs = keras.layers.Dense(num_outputs, activation="softmax")(x) # Use activation=softmax for classification, and activation=None for regression.
model = keras.Model(inputs, outputs)

model.summary()

83697664/83683744 [==============================] - 40s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 normalization (Normalizatio  (None, 150, 150, 3)      7         
 n)                                                              
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [ ]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

epochs = 100
model.fit(train_ds, epochs=epochs,validation_data = test_ds)

Epoch 1/100


C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


  7/255 [..............................] - ETA: 3:54 - loss: 5.8367 - sparse_categorical_accuracy: 0.0045 

Fine-Tune model

In [ ]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(5e-5), # 1e-5 was initially
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

epochs = 100
model.fit(train_ds, epochs = epochs, validation_data = test_ds)